In [ ]:
import tarfile
tf = tarfile.open("test_verification.tar")
tf.extractall()

In [ ]:
# import tarfile
tf = tarfile.open("validation_verification.tar")
tf.extractall()

In [1]:
import os
import numpy as np
from PIL import Image

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import time
import re
import csv

In [2]:
test_file = open('./test_trials_verification_student.txt', 'r')
A = test_file.readlines()
B = [i.split('\n')[0].split(' ') for i in A]

In [3]:
Path = './test_verification/'
class ImageDataset(Dataset):
    def __init__(self, file_list, target_list=None):
        self.file_list = file_list

        if target_list:
            self.target_list = target_list
            self.n_class = len(list(set(target_list)))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        
        path1, path2 = self.file_list[index]
        
        im1 = Image.open(Path + path1)
        img1 = torchvision.transforms.ToTensor()(im1)

        im2 = Image.open(Path + path2)
        img2 = torchvision.transforms.ToTensor()(im2)
        
        return img1,img2

In [4]:
# torch.nn.CosineSimilarity?
test_set = ImageDataset(B)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=1)

In [5]:
class ConvBNReLU(nn.Sequential):
    def __init__(self, in_planes, out_planes, kernel_size=3, stride=1, groups=1):
        padding = (kernel_size - 1) // 2
        super(ConvBNReLU, self).__init__(
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding, groups=groups, bias=False),
            nn.BatchNorm2d(out_planes),
            nn.ReLU6(inplace=True)
        )


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = int(round(inp * expand_ratio))
        self.use_res_connect = self.stride == 1 and inp == oup

        layers = []
        if expand_ratio != 1:
            # pw
            layers.append(ConvBNReLU(inp, hidden_dim, kernel_size=1))
        layers.extend([
            # dw
            ConvBNReLU(hidden_dim, hidden_dim, stride=stride, groups=hidden_dim),
            # pw-linear
            nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
            nn.BatchNorm2d(oup),
        ])
        self.conv = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)

In [17]:
class MobileNetV2(nn.Module):
    def __init__(self, input_channel, num_classes, width_mult=1.0, inverted_residual_setting=None):
        
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        
        if inverted_residual_setting is None:
            inverted_residual_setting = [
                # t, c, n, s
                [1, 64, 2, 2],
                [6, 128, 4, 2],
                [6, 256, 6, 1],
            ]

        self.last_channel = 1280
        features = []
        
        # building inverted residual blocks
        for t, c, n, s in inverted_residual_setting:
            output_channel = c
            for i in range(n):
                stride = s if i == 0 else 1
                features.append(block(input_channel, output_channel, stride, expand_ratio=t))
                input_channel = output_channel
                
        # building last several layers
        features.append(ConvBNReLU(input_channel, self.last_channel, kernel_size=1))
        
        # make it nn.Sequential
        self.features = nn.Sequential(*features)

        # building classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.last_channel, num_classes),
        )

        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.features(x)
#         print(x.shape)
        x1 = x.mean([2, 3])
        x3 = self.classifier(x1)
#         x2 = x1/torch.norm(self)
        x2 = x3/torch.norm(self.classifier[-1].weight)
        return x2


In [18]:
# save only model parameters
PATH = "./new_100.pt"
model = MobileNetV2(3, 2300)
model.load_state_dict(torch.load(PATH))
model

MobileNetV2(
  (features): Sequential(
    (0): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3, bias=False)
          (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace)
        )
        (1): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace)
        )
        (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [19]:
# model.classifier = nn.Sequential(*list(model.classifier.children())[:-2])

In [20]:
# def test_verify(model, test_loader):
#     model.eval()
#     test_loss = []
#     accuracy = 0
#     total = 0

#     for batch_num, (feats, labels) in enumerate(test_loader):
#         feats, labels = feats.to(device), labels.to(device)
#         outputs = model(feats)
        


In [21]:
import csv

In [22]:
results=[]

similar=torch.nn.CosineSimilarity()

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
model.eval()
model.to(device)
fout = open('valid_submission_4.csv', 'w')
fout.write('trial, score')
fout.write('\n')

for i,(img1,img2) in enumerate(test_loader):

    img1 = img1.to(device)
    img2 = img2.to(device)

    out1 = model(img1)
    out2 = model(img2)
    
#     print(out1, out2)
#     print(out2.shape)
    
    cos_sim=similar(out1,out2).cpu().detach().numpy()
    fout.write(B[i][0]+' '+B[i][1])
    fout.write(',')
    fout.write(str(cos_sim[0]))
    fout.write('\n')
    
    if i%10000==0:
        print(i, cos_sim)
        
#     print(cos_sim)
#     break
    
fout.close()

0 [0.44391257]
10000 [-0.26947072]
20000 [0.08183454]
30000 [0.51615185]
40000 [0.25812194]
50000 [0.02776192]
60000 [0.24209948]
70000 [-0.20362747]
80000 [0.2716744]
90000 [0.09897883]
100000 [0.08804475]
110000 [-0.0353479]
120000 [0.0229971]
130000 [-0.0411483]
140000 [0.09678245]
150000 [0.15870436]
160000 [0.17635518]
170000 [0.08102635]
180000 [0.36841297]
190000 [-0.13476355]
200000 [0.07647083]
210000 [0.2858794]
220000 [0.05896786]
230000 [0.04030159]
240000 [0.07953086]
250000 [0.23357236]
260000 [0.25332496]
270000 [0.11404207]
280000 [0.12073968]
290000 [0.16862662]
300000 [0.24041018]
310000 [0.4385024]
320000 [0.21011345]
330000 [0.4473296]
340000 [0.5598765]
350000 [0.14357041]
360000 [-0.11382651]
370000 [-0.0515466]
380000 [0.1440044]
390000 [-0.06863102]
400000 [0.42717978]
410000 [0.0098023]
420000 [0.2086842]
430000 [0.1298382]
440000 [-0.10465484]
450000 [-0.10366272]
460000 [0.23527181]
470000 [0.09115069]
480000 [0.50458163]
490000 [-0.1958339]
500000 [0.1301791

KeyboardInterrupt: 